Data taken from: https://www.statistik-nord.de/fileadmin/maps/zensus2011_hh/index.html

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import folium
import branca

: 

In [ ]:
path_census_data = r"C:\Users\srude\Documents\Dev Kram\Gravity_Model\Data\Zensus2011_Ergebnisse_nach_Gitterzellen_fuer_Hamburg.xlsx"

In [ ]:
df_census = pd.read_excel(
    path_census_data, sheet_name="Einwohnerzahl 100m-Gitterzellen", header=2
)

In [ ]:
df_census=df_census.drop([76039, 76038])

In [ ]:
# for testing purposes we will only take 500
#df_census = df_census[0:500]

In [ ]:
# for each row, determine latitude and longitude points
# for each cell, build polygon
# append polygon to list of polygons

ls_polygon_geometries = []
for i in range(len(df_census)):
    lon_point_list = [df_census.iloc[i,3] - 50, df_census.iloc[i,3] - 50, df_census.iloc[i,3] + 50, df_census.iloc[i,3] + 50]
    lat_point_list = [df_census.iloc[i,4] -50, df_census.iloc[i,4] + 50, df_census.iloc[i,4] + 50, df_census.iloc[i,4] - 50, ]
    print(df_census.iloc[i,2], df_census.iloc[i,3],df_census.iloc[i,4]  )
    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    ls_polygon_geometries.append(polygon_geom)

In [ ]:
crs = 'epsg:3035'
gdf_census = gpd.GeoDataFrame(df_census, crs=crs, geometry=ls_polygon_geometries)

In [ ]:
print(gdf_census['Einwohner'].max())

Colorbrewer: YlOrRd_09: https://jiffyclub.github.io/palettable/colorbrewer/sequential/
Styling Options: https://i.stack.imgur.com/dHNzs.png

In [ ]:
(gdf_census.centroid.x).min()
(gdf_census.centroid.y).min()

In [ ]:
colormap = branca.colormap.linear.YlOrRd_09.scale(0, 500)#gdf_census['Einwohner'].max() )
colormap.caption = "Einwohner"

# standard crs of folium maps: epsg 4326
map_hamburg = folium.Map([53.5460, 10.0003] , zoom_start=10,  tiles='cartodbpositron')
folium.GeoJson(gdf_census, tooltip=folium.GeoJsonTooltip(fields=["Gitter_ID_100m","Einwohner"]),  style_function= lambda x:{
        "stroke": True,
        "color": '#D3D3D3',
        "weight": 0.1,
        "lineOpacity": 0.5,
        "fillOpacity": 0.8,
        "fillColor":  "#FFFFF3" if x['properties']["Einwohner"] == -1 else colormap(x['properties']["Einwohner"] ),}).add_to(map_hamburg)
#folium.LatLngPopup().add_to(m)
colormap.add_to(map_hamburg)
map_hamburg

In [ ]:
gdf_census.to_file(filename=r'C:\Users\srude\Documents\Dev Kram\Gravity_Model_neu\Shapefiles\population_data_Hamburg_2011.shp', driver="ESRI Shapefile")